# Chapter 38: Monorepo vs. Polyrepo

The organizational structure of code repositories fundamentally shapes how teams collaborate, how continuous integration pipelines execute, and how dependencies flow through a microservices architecture. This architectural decision—whether to consolidate all code into a single repository (monorepo) or distribute it across multiple repositories (polyrepo)—impacts build performance, access control, versioning strategies, and developer workflow efficiency. Neither approach is universally superior; each optimizes for different organizational constraints, team scales, and deployment patterns.

This chapter examines the structural, technical, and organizational implications of monorepo and polyrepo strategies. We analyze build tooling optimizations for massive codebases, dependency management patterns that prevent version drift, CI pipeline designs that handle sparse changes efficiently, and governance models that maintain code quality across organizational boundaries. By understanding the trade-offs between atomic commits across services and independent repository lifecycles, organizations can select the appropriate structure for their specific context.

## 38.1 Monorepo Strategies

A monorepo consolidates all microservices, shared libraries, infrastructure code, and documentation into a single version control repository. This approach, famously employed by Google, Meta, and Netflix, enables atomic changes across multiple services while introducing challenges in build performance and access control.

### Repository Structure

A well-organized monorepo separates concerns through directory conventions:

```
monorepo/
├── .github/
│   └── workflows/           # Centralized or distributed CI configs
├── apps/
│   ├── payment-service/     # Individual microservices
│   │   ├── src/
│   │   ├── Dockerfile
│   │   ├── package.json
│   │   └── README.md
│   ├── order-service/
│   └── user-service/
├── libs/
│   ├── shared-kernel/       # Domain primitives
│   ├── event-contracts/     # Shared protobuf/avro schemas
│   └── observability/       # Shared logging/metrics libs
├── infra/
│   ├── terraform/           # Infrastructure as Code
│   └── k8s-manifests/       # Base Kubernetes configs
├── docs/
│   └── architecture/        # ADRs (Architecture Decision Records)
└── tools/
    └── scripts/             # Build automation scripts
```

**Key Organizational Principles:**
- **Apps**: Contain deployable units (microservices) with independent CI/CD pipelines
- **Libs**: Shared code consumed by multiple services but not deployed independently
- **Infra**: Cross-cutting infrastructure definitions
- **Docs**: Living documentation versioned with code

### Atomic Commits Across Services

The primary advantage of monorepos is the ability to modify multiple services in a single commit:

```bash
# Example: Adding a new field to the Payment API affects multiple services
git diff HEAD~1 --name-only

apps/payment-service/src/main/java/com/company/payment/PaymentRequest.java
apps/order-service/src/main/java/com/company/order/PaymentClient.java
libs/event-contracts/proto/payment_events.proto
infra/terraform/api-gateway/payment_routes.tf
```

**Commit Message Convention:**
```
feat(payment): Add installment payment option

- Update PaymentRequest to support installment plans
- Modify Order Service to calculate installment schedules
- Update API Gateway routes for new endpoints
- Add event schema for installment.created

BREAKING CHANGE: PaymentRequest now requires paymentType field
Refs: JIRA-1234
```

**Explanation:**
This commit touches four components atomically. If any part fails CI, the entire commit is rejected, preventing partial deployments that could leave the system in an inconsistent state. The breaking change notice triggers semantic versioning analysis in CI.

### Sparse Checkouts

For developers working on specific services, Git sparse checkouts reduce clone time and working directory size:

```bash
# Configure sparse checkout
git sparse-checkout init --cone

# Only checkout payment-service and shared libs
git sparse-checkout set apps/payment-service libs/shared-kernel

# Result: Working directory contains only these paths
# Other services exist in Git history but not locally
```

**Explanation:**
Sparse checkout uses the "cone" mode (directory-based) to include only specified directories. Developers working exclusively on Payment Service don't need Order Service code locally, reducing disk usage and IDE indexing time while maintaining the ability to commit cross-service changes when needed.

## 38.2 Polyrepo Strategies

Polyrepos distribute microservices across independent repositories, typically one per service or one per team. Each repository maintains independent versioning, CI/CD pipelines, and release cycles.

### Repository Structure

```
payment-service-repo/
├── .github/workflows/
├── src/
├── tests/
├── Dockerfile
├── docker-compose.yml
├── package.json
└── README.md

order-service-repo/
├── .github/workflows/
├── src/
├── tests/
└── ...

shared-contracts-repo/       # Shared schemas/interfaces
├── proto/
├── openapi/
└── ...
```

### Semantic Versioning for Dependencies

Polyrepos rely heavily on semantic versioning to manage cross-service dependencies:

```json
// order-service/package.json
{
  "name": "order-service",
  "version": "2.1.0",
  "dependencies": {
    "@company/event-contracts": "^1.2.0",
    "@company/observability": "~3.0.5",
    "axios": "^1.6.0"
  }
}
```

**Version Constraint Explanation:**
- `^1.2.0`: Compatible with 1.2.0 and above, but less than 2.0.0 (allows minor and patch updates)
- `~3.0.5`: Approximately equivalent to 3.0.5, allows only patch updates (3.0.x)
- Exact versions (`1.2.0`): Pinned to specific version for reproducibility

**Lock Files:**
```json
// package-lock.json (excerpt)
{
  "name": "@company/event-contracts",
  "version": "1.2.3",
  "resolved": "https://npm.company.com/@company/event-contracts/-/event-contracts-1.2.3.tgz",
  "integrity": "sha512-abc123...",
  "dependencies": {
    "protobufjs": "^7.2.0"
  }
}
```

**Explanation:**
Lock files (package-lock.json, yarn.lock, poetry.lock) freeze the entire dependency tree to specific versions. When CI checks out the code, it installs exactly the versions specified in the lock file, ensuring reproducible builds even if upstream packages release new versions.

### Submodule and Subtree Patterns

For shared code, polyrepos use Git submodules or subtrees:

```bash
# Adding shared contracts as submodule
git submodule add https://github.com/company/shared-contracts.git contracts/

# .gitmodules file created:
[submodule "contracts"]
    path = contracts
    url = https://github.com/company/shared-contracts.git
    branch = main
```

**CI Update Script:**
```yaml
# .github/workflows/update-contracts.yml
name: Update Contracts
on:
  schedule:
    - cron: '0 2 * * *'  # Daily at 2 AM

jobs:
  update:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
        with:
          submodules: recursive
      
      - name: Update Submodule
        run: |
          git submodule update --remote --merge
          if [[ -n $(git status --porcelain) ]]; then
            git add contracts/
            git commit -m "chore: Update shared contracts to latest"
            git push origin HEAD:main
          fi
```

**Explanation:**
This workflow automatically updates the submodule reference daily. If new commits exist in the shared-contracts repository, the submodule pointer updates and commits the change, triggering the service's CI pipeline to test against the latest contracts.

## 38.3 Build Tooling

Build tooling requirements differ dramatically between monorepos and polyrepos, particularly regarding incremental builds and change detection.

### Monorepo Build Tools

**Nx (JavaScript/TypeScript):**
Nx provides intelligent caching and task orchestration for JavaScript monorepos.

```json
// nx.json - Root configuration
{
  "extends": "nx/presets/npm.json",
  "npmScope": "company",
  "tasksRunnerOptions": {
    "default": {
      "runner": "nx/tasks-runners/default",
      "options": {
        "cacheableOperations": ["build", "test", "lint"],
        "parallel": 3
      }
    }
  },
  "namedInputs": {
    "default": ["{projectRoot}/**/*", "sharedGlobals"],
    "production": [
      "default",
      "!{projectRoot}/**/*.spec.ts",
      "!{projectRoot}/**/test/**/*"
    ]
  },
  "targetDefaults": {
    "build": {
      "dependsOn": ["^build"],
      "inputs": ["production", "^production"]
    }
  }
}
```

**Explanation:**
This configuration defines:
- **Cacheable Operations**: Results of build, test, and lint are cached in `node_modules/.cache/nx`
- **Named Inputs**: `production` excludes test files from build hashes
- **Target Defaults**: `dependsOn: ["^build"]` ensures dependencies build first (the `^` indicates upstream dependencies)

**Project Configuration:**
```json
// apps/payment-service/project.json
{
  "name": "payment-service",
  "sourceRoot": "apps/payment-service/src",
  "targets": {
    "build": {
      "executor": "@nx/node:webpack",
      "outputs": ["{options.outputPath}"],
      "options": {
        "outputPath": "dist/apps/payment-service",
        "main": "apps/payment-service/src/main.ts",
        "tsConfig": "apps/payment-service/tsconfig.app.json",
        "assets": ["apps/payment-service/src/assets"]
      }
    },
    "test": {
      "executor": "@nx/jest:jest",
      "outputs": ["{workspaceRoot}/coverage/apps/payment-service"],
      "options": {
        "jestConfig": "apps/payment-service/jest.config.ts"
      }
    },
    "docker": {
      "executor": "nx:run-commands",
      "options": {
        "commands": [
          "docker build -t payment-service:latest -f apps/payment-service/Dockerfile ."
        ]
      }
    }
  }
}
```

**Running Affected Commands:**
```bash
# Only build projects affected by current branch changes
nx affected --target=build --base=main --head=HEAD

# Result: Builds only payment-service and order-service if 
# shared-kernel changed and both depend on it
```

**Explanation:**
`nx affected` analyzes the Git diff between `main` and `HEAD`, builds a dependency graph, and executes targets only for projects with modified source files or dependencies. This prevents rebuilding the entire monorepo when only one service changes.

**Bazel (Multi-language):**
Bazel provides hermetic, reproducible builds across languages with sophisticated caching.

```python
# BUILD.bazel (Java service)
load("@rules_java//java:defs.bzl", "java_binary", "java_library")
load("@rules_docker//java:image.bzl", "java_image")

java_library(
    name = "payment_lib",
    srcs = glob(["src/main/java/**/*.java"]),
    deps = [
        "//libs/shared-kernel:shared_kernel",
        "@maven//:com_google_guava_guava",
        "@maven//:org_springframework_boot_spring_boot_starter_web",
    ],
    resources = glob(["src/main/resources/**"]),
)

java_binary(
    name = "payment_service",
    main_class = "com.company.payment.Application",
    runtime_deps = [":payment_lib"],
)

# Docker image generation
java_image(
    name = "payment_image",
    base = "@java_base//image",
    binary = ":payment_service",
)
```

**Explanation:**
- `java_library` compiles source files with dependencies on the shared kernel and external Maven artifacts
- `java_binary` creates an executable JAR with the specified main class
- `java_image` (from rules_docker) builds a minimal Docker image containing only the JAR and its runtime dependencies

**Bazel Query for CI:**
```bash
# Find all tests affected by changes to shared-kernel
bazel query 'rdeps(//..., //libs/shared-kernel:shared_kernel)' --output=label_kind | grep "java_test"

# Build only affected targets
bazel build $(bazel query 'set($(git diff --name-only main))' | grep BUILD | xargs dirname)
```

**Turborepo (JavaScript):**
Turborepo provides lightweight monorepo task running with aggressive caching.

```json
// turbo.json
{
  "$schema": "https://turbo.build/schema.json",
  "globalDependencies": ["**/.env.*local"],
  "pipeline": {
    "build": {
      "dependsOn": ["^build"],
      "outputs": [".next/**", "!.next/cache/**", "dist/**"]
    },
    "test": {
      "dependsOn": ["build"],
      "inputs": ["src/**/*.tsx", "src/**/*.ts", "test/**/*.ts"],
      "outputs": ["coverage/**"]
    },
    "deploy": {
      "dependsOn": ["build", "test", "lint"],
      "outputs": []
    },
    "lint": {}
  }
}
```

**Explanation:**
Turborepo's pipeline defines task dependencies. The `^` in `^build` indicates that a package's build task depends on its dependencies' build tasks first. Outputs are cached based on input file hashes, enabling instant replay of builds when no source files change.

### Polyrepo Build Optimization

In polyrepos, each repository maintains its own build chain, but shared libraries require publishing artifacts to a registry.

**GitHub Actions with Artifact Publishing:**
```yaml
# .github/workflows/build.yml
name: Build and Publish
on:
  push:
    branches: [main]

jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      
      - name: Setup Java
        uses: actions/setup-java@v3
        with:
          java-version: '17'
          distribution: 'temurin'
      
      - name: Build
        run: ./mvnw clean package
      
      - name: Publish to GitHub Packages
        run: ./mvnw deploy -Dregistry=https://maven.pkg.github.com/company
        env:
          GITHUB_TOKEN: ${{ secrets.GITHUB_TOKEN }}
```

**Consumer Repository:**
```xml
<!-- pom.xml in order-service -->
<dependencies>
    <dependency>
        <groupId>com.company</groupId>
        <artifactId>payment-client</artifactId>
        <version>2.1.0</version>
    </dependency>
</dependencies>

<repositories>
    <repository>
        <id>github</id>
        <url>https://maven.pkg.github.com/company/payment-service</url>
    </repository>
</repositories>
```

## 38.4 Dependency Management

### Monorepo Dependency Strategies

**Internal Dependencies:**
In monorepos, internal dependencies reference source code directly rather than published artifacts:

```json
// apps/order-service/package.json
{
  "name": "@company/order-service",
  "dependencies": {
    "@company/shared-kernel": "*",  // Symlink to libs/shared-kernel
    "@company/event-contracts": "*",
    "express": "^4.18.0"
  }
}
```

**Workspace Configuration (npm/pnpm/yarn):**
```json
// package.json (root)
{
  "private": true,
  "workspaces": [
    "apps/*",
    "libs/*"
  ]
}
```

**Explanation:**
The `*` version indicates the latest version in the monorepo. Package managers (npm, yarn, pnpm) create symlinks from `node_modules/@company/shared-kernel` to `libs/shared-kernel`, allowing changes to shared libraries to immediately reflect in consuming services without publishing.

**Dependency Graph Visualization:**
```bash
# Nx dependency graph
nx graph

# Outputs interactive visualization showing:
# - payment-service depends on shared-kernel and event-contracts
# - order-service depends on payment-client (in payment-service) and shared-kernel
# - Circular dependencies highlighted in red
```

### Polyrepo Dependency Management

**Version Alignment Strategy:**
Maintain a shared dependency versions file:

```json
// shared-dependencies.json (published as artifact)
{
  "versions": {
    "spring-boot": "3.2.0",
    "postgresql": "15.4",
    "kafka": "3.5.0",
    "junit": "5.10.0"
  }
}
```

**Consumer Implementation:**
```groovy
// build.gradle (Gradle)
plugins {
    id 'java'
    id 'org.springframework.boot' version '3.2.0'  // Manually aligned
}

dependencies {
    implementation platform('com.company:shared-bom:1.0.0')
    // Versions managed by BOM (Bill of Materials)
    implementation 'org.springframework.boot:spring-boot-starter-web'
    implementation 'org.postgresql:postgresql'
}
```

**Explanation:**
A BOM (Bill of Materials) is a special POM that manages dependency versions centrally. By importing the shared BOM, services inherit version constraints without specifying versions explicitly, ensuring consistency across the organization.

## 38.5 CI Pipeline Design

### Monorepo CI Strategies

**Change Detection:**
Only run pipelines for modified projects:

```yaml
# .github/workflows/monorepo-ci.yml
name: Monorepo CI
on:
  push:
    branches: [main, develop]
  pull_request:
    branches: [main]

jobs:
  detect-changes:
    runs-on: ubuntu-latest
    outputs:
      payment-changed: ${{ steps.changes.outputs.payment }}
      order-changed: ${{ steps.changes.outputs.order }}
      shared-changed: ${{ steps.changes.outputs.shared }}
    steps:
      - uses: actions/checkout@v4
        with:
          fetch-depth: 0  # Full history for diff
      
      - name: Detect Changes
        id: changes
        run: |
          # Check which paths changed
          git diff --name-only HEAD~1 > changed-files.txt
          
          echo "payment=$(grep -q 'apps/payment-service/' changed-files.txt && echo 'true' || echo 'false')" >> $GITHUB_OUTPUT
          echo "order=$(grep -q 'apps/order-service/' changed-files.txt && echo 'true' || echo 'false')" >> $GITHUB_OUTPUT
          echo "shared=$(grep -q 'libs/' changed-files.txt && echo 'true' || echo 'false')" >> $GITHUB_OUTPUT

  test-payment:
    needs: detect-changes
    if: ${{ needs.detect-changes.outputs.payment-changed == 'true' || needs.detect-changes.outputs.shared-changed == 'true' }}
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      
      - name: Setup Nx
        uses: nrwl/nx-set-shas@v3
      
      - name: Test Payment Service
        run: nx test payment-service --skip-nx-cache

  test-order:
    needs: detect-changes
    if: ${{ needs.detect-changes.outputs.order-changed == 'true' || needs.detect-changes.outputs.shared-changed == 'true' }}
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      
      - name: Test Order Service
        run: nx test order-service
```

**Explanation:**
The `detect-changes` job uses `git diff` to determine which paths modified. Downstream jobs use `if` conditions to only run when relevant code changes. If `libs/shared-kernel` changes, both payment and order service tests run because they depend on the shared library.

**Matrix Strategy for Scale:**
```yaml
  test-affected:
    runs-on: ubuntu-latest
    strategy:
      matrix:
        project: ${{ fromJson(needs.detect-changes.outputs.affected-projects) }}
    steps:
      - uses: actions/checkout@v4
      - run: nx test ${{ matrix.project }}
```

### Polyrepo CI Strategies

**Cross-Repository Triggers:**
When shared libraries change, trigger downstream service builds:

```yaml
# shared-contracts/.github/workflows/trigger-downstream.yml
name: Trigger Downstream Builds
on:
  push:
    branches: [main]

jobs:
  notify:
    runs-on: ubuntu-latest
    steps:
      - name: Trigger Order Service Build
        uses: peter-evans/repository-dispatch@v2
        with:
          token: ${{ secrets.CROSS_REPO_TOKEN }}
          repository: company/order-service
          event-type: contract-updated
          client-payload: '{"version": "${{ github.sha }}", "ref": "${{ github.ref }}"}'
```

**Consumer Handling:**
```yaml
# order-service/.github/workflows/build.yml
name: Build
on:
  push:
    branches: [main]
  repository_dispatch:
    types: [contract-updated]

jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      
      - name: Update Contracts
        if: github.event.action == 'contract-updated'
        run: |
          npm install @company/contracts@${{ github.event.client_payload.version }}
      
      - name: Run Tests
        run: npm test
```

**Explanation:**
When the shared-contracts repository updates, it sends a repository dispatch event to order-service. The order-service workflow detects this event type and updates the dependency to the new version before running tests, ensuring compatibility with the latest contracts.

## 38.6 Code Ownership

### Monorepo CODEOWNERS

GitHub/GitLab CODEOWNERS files define mandatory reviewers for specific paths:

```ini
# CODEOWNERS (monorepo root)
# Global fallback
* @company/platform-team

# Service-specific ownership
/apps/payment-service/** @company/payment-team @alice @bob
/apps/order-service/** @company/order-team @charlie

# Shared libraries require broader review
/libs/shared-kernel/** @company/architects @company/platform-team
/libs/event-contracts/** @company/architects

# Infrastructure changes
/infra/terraform/** @company/sre-team
/infra/k8s-manifests/** @company/sre-team @company/security-team

# CI/CD configuration
/.github/workflows/** @company/platform-team @company/security-team

# Sensitive configuration
/**/secrets.yaml @company/security-leads
```

**Explanation:**
- Lines starting with `#` are comments
- Patterns match file paths (glob syntax)
- Multiple teams/users can be specified
- Last matching pattern takes precedence
- Protected branches can require CODEOWNER approval before merging

### Polyrepo Access Control

In polyrepos, repository-level permissions provide stronger isolation:

```bash
# GitHub CLI configuration
gh repo create company/payment-service --private --team payment-team

# Set permissions
gh api repos/company/payment-service/collaborators/payment-team \
  -X PUT \
  -f permission=admin

# External teams get read-only access
gh api repos/company/payment-service/collaborators/order-team \
  -X PUT \
  -f permission=pull
```

**Branch Protection Rules:**
```yaml
# .github/settings.yml (Probot Settings)
branches:
  - name: main
    protection:
      required_pull_request_reviews:
        required_approving_review_count: 2
        require_code_owner_reviews: true
        dismissal_restrictions:
          users: []
          teams: [payment-team]
      required_status_checks:
        strict: true
        contexts: ["ci/tests", "ci/security-scan"]
      enforce_admins: false
      restrictions:
        users: []
        teams: [payment-team]
```

**Explanation:**
Branch protection ensures:
- 2 approving reviews required
- CODEOWNERS must approve (ensuring domain experts review)
- Status checks (CI tests, security scans) must pass
- Only payment-team members can push to main

## 38.7 Pros and Cons

### Monorepo Advantages

**1. Atomic Changes**
Cross-service refactoring is a single commit:
```bash
git checkout -b refactor-payment-model
# Edit 5 services and shared libs
git commit -m "refactor: Unify payment models across services"
git push
```
No versioning coordination between repositories required.

**2. Single Source of Truth**
One Git history shows exactly which changes deployed together:
```bash
git log --oneline --graph --all --decorate
# Shows commit touched payment, order, and inventory simultaneously
```

**3. Shared Tooling**
One version of build tools, linters, and test frameworks across all projects.

**4. Code Reuse**
Easy to extract shared libraries:
```bash
mkdir libs/new-shared-module
git mv apps/payment-service/src/shared/* libs/new-shared-module/
# Update imports in same commit
```

### Monorepo Disadvantages

**1. Scale Challenges**
Large repositories cause:
- Slow Git operations (status, log, clone)
- Large CI runner storage requirements
- IDE indexing performance degradation

**Mitigation:**
- Partial clones: `git clone --filter=blob:none`
- Virtual file systems (Git VFS, EdenFS)
- Distributed build caching (Bazel Remote Cache)

**2. Permission Complexity**
Cannot restrict access to specific directories easily. All engineers see all code (though this is often considered a feature).

**3. Coupling Risk**
Easy to create tight coupling between services through shared code changes.

### Polyrepo Advantages

**1. Strong Isolation**
Services are truly independent:
```bash
# Order service repo knows nothing of Payment service internals
rm -rf payment-service/  # Cannot accidentally delete other service
```

**2. Independent Release Cycles**
Payment Service can release v3.0.0 while Order Service remains on v2.x without monorepo coordination overhead.

**3. Technology Diversity**
Different repositories can use different languages without build tool conflicts:
- payment-service: Java with Maven
- order-service: Node.js with npm
- analytics-service: Python with Poetry

**4. Access Control**
Precise repository-level permissions prevent accidental cross-service modifications.

### Polyrepo Disadvantages

**1. Cross-Service Refactoring Complexity**
Changing a shared API requires:
1. Update shared library repository
2. Publish new version
3. Update Service A repository
4. Update Service B repository
5. Coordinate deployment timing

**2. Version Drift**
Services may use different versions of shared libraries, causing integration issues:
```json
// Service A uses contracts v1.2.0
// Service B uses contracts v1.3.0 (breaking change)
// Integration fails in production despite both passing CI
```

**3. Code Duplication**
Without easy sharing, utilities get duplicated across repositories.

## 38.8 Choosing the Right Approach

### Decision Matrix

| Factor | Monorepo | Polyrepo |
|--------|----------|----------|
| **Team Size** | >100 engineers | <50 engineers |
| **Service Count** | >50 microservices | <20 services |
| **Change Frequency** | High cross-service changes | Independent evolution |
| **Compliance** | Unified audit trail | Per-service isolation |
| **Build Tool** | Bazel, Nx, Rush | Standard per-language tools |
| **Deployment** | Coordinated releases | Fully independent |

### Hybrid Approaches

**Monorepo with Polyrepo Artifacts:**
Source in monorepo, deploy artifacts to registries:
```bash
# Monorepo builds service
nx build payment-service

# Publishes Docker image to registry
docker push company/payment-service:$(git rev-parse HEAD)

# Other services consume via registry, not source
```

**Polyrepo with Monorepo Tooling:**
Use git submodules or meta-repos to simulate monorepo workflows:
```bash
# Meta-repository containing submodules
meta/
├── payment-service (submodule)
├── order-service (submodule)
└── shared-contracts (submodule)

# Batch operations across repos
meta exec "git checkout main && git pull"
```

### Migration Strategies

**Polyrepo to Monorepo:**
Use tools like `tomono` or `git subtree` to preserve history:
```bash
# Preserve all polyrepo histories in monorepo
git subtree add --prefix=apps/payment-service payment-repo main
git subtree add --prefix=apps/order-service order-repo main
```

**Monorepo to Polyrepo:**
Use `git filter-branch` to extract service history:
```bash
# Extract payment-service to new repo
git filter-branch --prune-empty --subdirectory-filter apps/payment-service \
  -- --all
```

---

## Chapter Summary and Preview

This chapter analyzed the structural foundations of microservices development through the lens of repository organization strategies. Monorepos consolidate all services into a single version control repository, enabling atomic commits across service boundaries and facilitating large-scale refactoring through tools like Nx, Bazel, and Turborepo that provide intelligent change detection and build caching. However, this approach introduces challenges in access control granularity, repository scaling, and the potential for tight coupling through excessive shared code dependencies.

Polyrepos distribute services across independent repositories, providing strong isolation boundaries, independent release cycles, and natural technology heterogeneity. This approach excels in organizations prioritizing service autonomy and team independence but introduces friction in cross-service refactoring, version drift risks in shared dependencies, and the operational overhead of coordinating changes across repository boundaries through semantic versioning and artifact publishing.

We examined build tooling optimizations essential for monorepo scale, including sparse checkouts to reduce working directory size, affected commands to minimize CI execution to changed components, and remote caching to share build artifacts across developer machines. For polyrepos, we explored Bill of Materials (BOM) patterns for version alignment and cross-repository triggering mechanisms to maintain integration integrity. Code ownership strategies through CODEOWNERS files and repository-level permissions provide governance appropriate to each model's access patterns.

**Key Takeaways:**
- Choose monorepos when your architecture requires frequent cross-service refactoring and you can invest in build tooling (Bazel/Nx) to handle scale; choose polyrepos when team autonomy and independent deployment velocity outweigh the costs of cross-repository coordination.
- In monorepos, implement affected commands and change detection to prevent CI pipeline explosion, ensuring only modified services and their dependents execute tests.
- In polyrepos, maintain strict semantic versioning discipline for shared libraries, utilizing automated dependency update tools (Renovate, Dependabot) to prevent version drift.
- Use CODEOWNERS in monorepos to enforce domain expert review despite open access to all code; use repository permissions in polyrepos to enforce architectural boundaries.
- Consider hybrid approaches (monorepo for shared contracts, polyrepo for implementations) to balance atomic change capability with deployment independence.

**Next Chapter Preview:**
Chapter 39: Multi-Environment Deployments explores strategies for managing configuration and deployment across development, staging, and production environments. We will examine environment promotion strategies that ensure artifact immutability while allowing configuration variance, techniques for managing environment-specific secrets and configurations without code duplication, and patterns for database migration coordination across environments. The chapter covers GitOps approaches to multi-environment management, drift detection between environments, and strategies for hotfixes and rollbacks when production issues arise, building upon the repository structures established in this chapter to create reliable promotion pipelines from development workstations to production clusters.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='37. microservices_cicd.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='39. multi_environment_deployments.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
